tgb - 7/15/2020 - Goal today is to determine the inputs and outputs of the real-geography NN and basic statistical properties to design the loss function

# Imports

In [10]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator

import tensorflow as tf
from tensorflow import keras as K
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# import pickle
# import sklearn
# from sklearn.linear_model import LinearRegression
# from scipy import stats

# Otherwise tensorflow will use ALL your GPU RAM for no reason
#limit_mem()

# Load real-geography dataset

## Open files 

In [3]:
pathref = '/DFS-L/DATA/pritchard/liranp/SST_m0001Kp_Neuralnet_SPCAM_216/atm/hist/SST_m0001Kp_Neuralnet_SPCAM_216.cam.h1.20'
pathp4K = '/DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.20'
pathm4K = '/DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.20'

In [8]:
dataref = xr.open_mfdataset(pathref+'09-0*-23-00000.nc',decode_times=False)
datap4K = xr.open_mfdataset(pathp4K+'09-0*-23-00000.nc',decode_times=False)

/DFS-L/DATA/pritchard/tbeucler/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/DFS-L/DATA/pritchard/tbeucler/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from

In [9]:
datap4K

<xarray.Dataset>
Dimensions:          (ilev: 31, lat: 96, lev: 30, lon: 144, nbnd: 2, slat: 95, slon: 144, time: 144)
Coordinates:
  * lat              (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon              (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * slat             (slat) float64 -89.05 -87.16 -85.26 ... 85.26 87.16 89.05
  * ilev             (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * lev              (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * slon             (slon) float64 -1.25 1.25 3.75 6.25 ... 351.2 353.8 356.2
  * time             (time) float64 105.0 105.0 105.0 ... 348.1 348.1 348.2
Dimensions without coordinates: nbnd
Data variables:
    hyam             (time, lev) float64 dask.array<chunksize=(16, 30), meta=np.ndarray>
    hybm             (time, lev) float64 dask.array<chunksize=(16, 30), meta=np.ndarray>
    hyai             (time, ilev) float64 dask.array<chunksize=(16, 31), meta=np.ndarray>
    hybi             (time, ilev) float64 dask.array<chunksize=(16, 31), meta=np.ndarray>
    P0               (time) float64 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    date             (time) int32 dask.array<chunksize=(16,), meta=np.ndarray>
    datesec          (time) int32 dask.array<chunksize=(16,), meta=np.ndarray>
    w_stag           (time, slat) float64 dask.array<chunksize=(16, 95), meta=np.ndarray>
    time_bnds        (time, nbnd) float64 dask.array<chunksize=(16, 2), meta=np.ndarray>
    date_written     (time) |S8 dask.array<chunksize=(16,), meta=np.ndarray>
    time_written     (time) |S8 dask.array<chunksize=(16,), meta=np.ndarray>
    ntrm             (time) int32 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    ntrn             (time) int32 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    ntrk             (time) int32 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    ndbase           (time) int32 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    nsbase           (time) int32 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    nbdate           (time) int32 20081010 20081010 ... 20081010 20081010
    nbsec            (time) int32 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    mdt              (time) int32 900 900 900 900 900 ... 900 900 900 900 900
    nlon             (time, lat) int32 dask.array<chunksize=(16, 96), meta=np.ndarray>
    wnummax          (time, lat) int32 dask.array<chunksize=(16, 96), meta=np.ndarray>
    gw               (time, lat) float64 dask.array<chunksize=(16, 96), meta=np.ndarray>
    ndcur            (time) int32 dask.array<chunksize=(16,), meta=np.ndarray>
    nscur            (time) int32 dask.array<chunksize=(16,), meta=np.ndarray>
    co2vmr           (time) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    ch4vmr           (time) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    n2ovmr           (time) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    f11vmr           (time) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    f12vmr           (time) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    sol_tsi          (time) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    nsteph           (time) int32 dask.array<chunksize=(16,), meta=np.ndarray>
    CLDICEAP         (time, lev, lat, lon) float32 dask.array<chunksize=(16, 30, 96, 144), meta=np.ndarray>
    CLDLIQAP         (time, lev, lat, lon) float32 dask.array<chunksize=(16, 30, 96, 144), meta=np.ndarray>
    CLOUD            (time, lev, lat, lon) float32 dask.array<chunksize=(16, 30, 96, 144), meta=np.ndarray>
    CRM_VTEND        (time, lev, lat, lon) float32 dask.array<chunksize=(16, 30, 96, 144), meta=np.ndarray>
    FLDS             (time, lat, lon) float32 dask.array<chunksize=(16, 96, 144), meta=np.ndarray>
    FLNS             (time, lat, lon) float32 dask.array<chunksize=(16, 96, 144), meta=np.ndarray>
    FLNT             (time, lat, lon) float32 dask.array<chunksize=(16, 96, 144), meta=np.ndarray

## Inputs

### Absolute Temperature

In [15]:
datap4K.TBP

<xarray.DataArray 'TBP' (time: 144, lev: 30, lat: 96, lon: 144)>
dask.array<concatenate, shape=(144, 30, 96, 144), dtype=float32, chunksize=(16, 30, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) float64 105.0 105.0 105.0 105.0 ... 348.1 348.1 348.1 348.2
Attributes:
    mdims:      1
    units:      K
    long_name:  Temperature (before physics)

### Specific Humidity

In [16]:
datap4K.QBP

<xarray.DataArray 'QBP' (time: 144, lev: 30, lat: 96, lon: 144)>
dask.array<concatenate, shape=(144, 30, 96, 144), dtype=float32, chunksize=(16, 30, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) float64 105.0 105.0 105.0 105.0 ... 348.1 348.1 348.1 348.2
Attributes:
    mdims:      1
    units:      kg/kg
    long_name:  Specific humidity                                            ...

### Meridional velocity *before* physics

Question - Is DT 15 minutes here?

In [30]:
24*(datap4K.time[1].values-datap4K.time[0].values)

0.2500000000001137

In [32]:
DT = 15*60
VBP = datap4K.VAP-datap4K.CRM_VTEND*DT

### Surface pressure

In [34]:
datap4K.PS

<xarray.DataArray 'PS' (time: 144, lat: 96, lon: 144)>
dask.array<concatenate, shape=(144, 96, 144), dtype=float32, chunksize=(16, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) float64 105.0 105.0 105.0 105.0 ... 348.1 348.1 348.1 348.2
Attributes:
    units:      Pa
    long_name:  Surface pressure

### Solar insolation (at the top of the atmosphere)

In [36]:
datap4K.NN2L_SOLSD

<xarray.DataArray 'NN2L_SOLSD' (time: 144, lat: 96, lon: 144)>
dask.array<concatenate, shape=(144, 96, 144), dtype=float32, chunksize=(16, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) float64 105.0 105.0 105.0 105.0 ... 348.1 348.1 348.1 348.2
Attributes:
    units:      W/m2
    long_name:  Diffuse solar rad on surface (< 0.7)